In [1]:
import numpy as np
from numpy.lib.stride_tricks import sliding_window_view
import matplotlib.pyplot as plt
import pandas as pd

import datetime


import torch
import torch.nn as nn

# Transforming the Dataset

In [3]:
df = pd.read_csv("data/MTA_Subway_Hourly_Ridership_Jan_Through_May_2025.csv", low_memory=False)

This dataframe includes the number of riders, per station complex, per hour, for every hour in every day for the months January-April.

My goal is to see if I can create a model to forecast subway ridership. It should work like this: Given some ridership, that is the riders for every hour of a given time period, can I forecast the ridership in the near future? For a preliminary investigation, I will consider a lookback length of 24 hours, and a prediction horizon of 6 hours. 

Naturally, I need the ridership by hour for each 24 hour period, and the ridership of the next 6 hours, for each station, to construct my training dataset. The dataset as is needs to be transformed

Exogenous features to consider alongside time series:

- It may be important to account for the day of the week. For example, it will be more difficult to predict Saturday from Friday, than say Tuesday from Monday, due to the distinct nature of weekend and weekday ridership. 

- Furthermore, station information may need to be taken into consideration. Different stations experience different daily ridership patterns. Popular stations such as Times Square-42nd St. have a lot of ridership constantly, while some stations on the outer edges of the system may have little to no ridership some days.

    - Note! One way to go about this is to include the (normazlied) lat/long coordinates with datapoints. This is better than simply one-hot encoding by station_complex because lat/long coordinates will contain information of where stations are with respect to one another. In a previous project, I identified a strong correlation between daily ridership pattern (on weekends) and location.
      
    - Note! In this dataset, there are sometimes multiple lat/long pairs listed for a given station_complex. This phenomenon is demonstrated in lat_long_discrepancy/lat_long_discrepancy.ipynb . Because I believe this is an error, I will take the mean of the lats/longs per station_complex as that station's true lat/long for my purposes here.

In [6]:
# Creating a df which contains station_complex, station_complex_id, latitude, longitude
# so I can keep just station_complex_id for now, and later merge station_info_df with on=station_complex_id to 
# recover latitudes and longitudes.

# As mentioned above, I take the mean(latitude) and mean(longitude) per station_complex.

station_info_df = df[df["transit_mode"] == "subway"][['station_complex_id', 'latitude', 'longitude']].drop_duplicates().reset_index(drop= True)
station_info_df = station_info_df.groupby(["station_complex_id"], as_index = False).agg({"latitude": 'mean', "longitude": 'mean'})

In [12]:
# Normalizing the latitudes and longitudes to zero mean and unit variance.

station_info_df['latitude'] = (station_info_df['latitude'] - station_info_df['latitude'].mean())/station_info_df['latitude'].std()
station_info_df['longitude'] = (station_info_df['longitude'] - station_info_df['longitude'].mean())/station_info_df['longitude'].std()

In [14]:
# Transforming df to a workable form

# Only considering Subway!
df = df[df["transit_mode"] == "subway"]

# Dropping Other Unneccesary Rows!
df = df.drop(["transfers", "Georeference", "station_complex", "latitude", "longitude", "borough"], axis = 1)

# Pandas datetime format.
df['transit_timestamp'] = pd.to_datetime(df['transit_timestamp'], format = "%m/%d/%Y %I:%M:%S %p")

# Keeping only date and hour, then discarding transit_timestamp
df['date'] = df['transit_timestamp'].dt.date
df['hour'] = df['transit_timestamp'].dt.hour
df.drop(["transit_timestamp"], axis = 1)

# Aggregating total ridership across different payment methods.
df = df.groupby(['station_complex_id', 'date', 'hour'], as_index = False).agg(ridership = pd.NamedAgg(column = "ridership", aggfunc = "sum"))


In [16]:
# If there were 0 riders for a certain station at a certain hour, that data is not recored into the dataset.
# Therefore, we need to fill in these values with 0 riders.

# First, create a df for the cartesian product of all possibles dates, hours, and station complexes.
all_stations = df[["station_complex_id"]].drop_duplicates().reset_index(drop=True)         # 424 station complexes
all_hours = pd.DataFrame({'hour':[i for i in range(24)]})                                  # 24 hours
all_dates = df[['date']].drop_duplicates().reset_index(drop = True)                        # Jan-Apr = 31 + 29 + 30 + 31 = 121 days

dates_hours = pd.merge(all_dates, all_hours, how = "cross")
dates_hours_stations = pd.merge(dates_hours, all_stations, how = "cross")
# dates_hours_stations contains every possible date, hour, and station complex in the recording period.

n_missing_values = dates_hours_stations.shape[0] - df.shape[0]

# Left join dates_hours_stations with the subway dataset, so that we have df with every date, hour, and station complex, with NaNs
# where there was no data recorded in the initial dataset.
df = pd.merge(dates_hours_stations, df, on = ["date", "hour", "station_complex_id"], how = "left")

# Replace NaN values with 0.
df = df.fillna(0)

print(f"{n_missing_values} missing entries detected for stations/hours with zero ridership.")


44145 missing entries detected for stations/hours with zero ridership.


# Generating Datapoints

The data will be split as follows:

- Training Data : 01/01/2025 - 03/31/25

- Validation Data : 04/01/2025 - 04/16/2025

- Testing Data : 04/17/2025 - 04/30/2025

In [19]:
# Training on points in the range [01/01/2025, 04/01/2025)
train_df = df[df["date"] < datetime.date(2025, 4, 1)]

# Validating on points in the range [04/01/2025, 04/17/2025)
val_df   = df[(df["date"] >= datetime.date(2025, 4, 1)) & (df["date"] < datetime.date(2025, 4, 17))]

# Testing on points in the range [04/17/2025, 05/01/2025)
test_df  = df[df["date"] >= datetime.date(2025, 4, 17)]


In [21]:
def get_X_y_from_df(inp_df, lookback = 24, prediction_horizon = 6):
    """
    Function for creating (X, y) datapoints from an input dataframe, where X = (latitude, longitude, r_{t}, ..., r_{t + lookback}) and 
    y = (r_{t + lookback + 1}, ..., r_{t + lookback + prediction_horizon - 1}), where r_{i} is the ridership at a given hour/day for 
    the station at the specified latitude and longitude.

    Inputs:
        inp_df: Pandas Dataframe object with columns ["station_complex_id", "date", "hour", "ridership"] where there are NO gaps in the
        date/hour datapoints.
        
        lookback: Size of lookback window in hours, default: 24.
        
        prediction_horizon: Size of prediction horizon in hours, default: 6.

    Outputs:
        X: Input datapoints
        y: 
    """

    def generate_data_points_by_station_complex(x):
        """
        Helper function for generating time series windows using numpy.lib.stride_tricks.sliding_window_view(), and adding them to
        the total data_points array.
        """

        # Make sure data is ordered so that it is a contnuous count of the ridership, i.e. each subsequent row is the following hour's
        # ridership., for every hour in the timeframe of inp_df, for this specific station.
        
        x = x.sort_values(by = ["date", "hour"])
        x.reset_index(drop=True, inplace=True)

        # Get latitude of this specific station, construct array of latitudes to concatenate in front of the time series window.
        lat = station_info_df[station_info_df["station_complex_id"] == x["station_complex_id"][0]]["latitude"].iloc[0]
        lats = np.full((x.shape[0] - lookback - prediction_horizon + 1, 1), lat)

        # Get longitude of this specific station, construct array of latitudes to concatenate in front of the time series window.
        long = station_info_df[station_info_df["station_complex_id"] == x["station_complex_id"][0]]["longitude"].iloc[0]
        longs = np.full((x.shape[0] - lookback - prediction_horizon + 1, 1), long)

        # Need numpy array for sliding_window_view()
        riderships_array = x["ridership"].to_numpy()
        
        # https://numpy.org/doc/stable/reference/generated/numpy.lib.stride_tricks.sliding_window_view.html
        window = sliding_window_view(riderships_array, lookback + prediction_horizon)

        # Add longitudes.
        window_longs = np.concatenate((longs, window), axis = 1)

        # Add latitudes.
        window_lat_longs = np.concatenate((lats, window_longs), axis = 1)
        
        # This function iteratively concatenates new data_points to the existing data_points array, constructed in the main function.
        nonlocal data_points
        data_points = np.concatenate((data_points, window_lat_longs), axis = 0)

    # Constructing empty array to store generated datapoints. Factor of 2 in size to account for latitude and longitude.
    data_points = np.empty((0, 2 + lookback + prediction_horizon))
    
    # Using groupby to generate separate data points for each station_complex
    inp_df.groupby(["station_complex_id"], as_index = False).apply(generate_data_points_by_station_complex)

    # First two points are latitude, longitude, next n = lookback points are the lookback window.
    X = data_points[:, :2 + lookback]

    # Next m = prediction_horizon points are the points to be predicted.
    y = data_points[:, 2 + lookback:]

    return X, y 

In [23]:
X_train, y_train = get_X_y_from_df(train_df, 24, 6)
X_test, y_test   = get_X_y_from_df(test_df, 24, 6)
X_val, y_val     = get_X_y_from_df(val_df, 24, 6)

/var/folders/x5/mt3y0q2n65zcpd0hz503pkx00000gn/T/ipykernel_92060/2489767023.py:60: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  inp_df.groupby(["station_complex_id"], as_index = False).apply(generate_data_points_by_station_complex)
/var/folders/x5/mt3y0q2n65zcpd0hz503pkx00000gn/T/ipykernel_92060/2489767023.py:60: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  inp_df.groupby(["station_complex_id"], as_index 

In [25]:
np.save("X_train.npy", X_train)
np.save("y_train.npy", y_train)

np.save("X_test.npy", X_test)
np.save("y_test.npy", y_test)

np.save("X_val.npy", X_val)
np.save("y_val.npy", y_val)